<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive
%load_ext Cython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [3]:
import numpy as np
from scipy.special import expit
import pandas as pd
import pickle
from numba import jit as njit
from functools import lru_cache
from multiprocessing import Pool
import os
import time
import random
import ta
from net import *
from memory import *
from reward import *
import traceback
from IPython.display import clear_output

In [0]:
def swish(x):
    x *= tf.nn.sigmoid(x)
    return x

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def net(x,layer):
  for i in layer:
    x = i(x)
  return x

def actor(x,action):
  with tf.variable_scope("actor", reuse=False):
    inputs = x
    with tf.variable_scope("main", reuse=False):
      x1 = tf.keras.layers.Conv1D(128,8,1,padding="causal",activation=swish)(x)
      x2 = tf.keras.layers.Conv1D(128,4,1,padding="causal",activation=swish)(x)
      x3 = tf.keras.layers.Conv1D(128,2,1,padding="causal",activation=swish)(x)
      x = tf.keras.layers.Concatenate()([x1,x2,x3])
      x = tf.keras.layers.Conv1D(128,3,1,padding="valid",activation=swish)(x)
      f = tf.keras.layers.Flatten()(x)
    with tf.variable_scope("sub", reuse=False):
      x = tf.keras.layers.Flatten()(inputs)
      shape = int(x.shape[-1])
      a = tf.keras.layers.Dense(shape,tf.nn.softmax)(f)
      mul = tf.keras.layers.Multiply()([x,a])
      out = tf.keras.layers.Dense(128,swish)(mul)
      out = tf.keras.layers.Dense(2,tf.tanh)(out)
    with tf.variable_scope("main2", reuse=False):
      # x = tf.keras.layers.Dense(128,tf.nn.relu)(f)
      x1 = tf.keras.layers.Concatenate()([out,f])
      x2 = tf.keras.layers.Concatenate()([action,f])
      layer = [tf.keras.layers.Dense(128,tf.nn.relu),tf.keras.layers.Dense(2)]
      x1 = net(x1,layer)
      x2 = net(x2,layer)
      X = tf.keras.layers.Maximum()([x1[:,0],x1[:,1]])
  return X,x1,x2,out

In [0]:
class Agent:
    def __init__(self,path,window_siz):
      self.path = path
      self.window_size = window_size
      self.STEP_SIZE = 20 * 6
      self.preproc()
      self.state_size = (None,self.window_size,self.df.shape[-1])
      print(self.state_size)
      self.memory = Memory(5000)
      self.rewards = reward3
      self.noise = 0.5
      self.noise_min = 0.1
      self.sess = tf.Session()

      with tf.variable_scope("input"):
        self.state = tf.placeholder(tf.float32, self.state_size)
        self.new_state = tf.placeholder(tf.float32, self.state_size)
        self.action = tf.placeholder(tf.float32,(None,2))
        self.reward = tf.placeholder(tf.float32,(None,2))

      with tf.variable_scope("model", reuse=False):
        self.policy,self.q_pi,self.q,self.out = actor(self.state,self.action)
      
      with tf.variable_scope("target", reuse=False):
        _,self.target_q_pi,_,_ = actor(self.new_state,self.action)

      with tf.variable_scope("loss"):
        self.loss = loss = (0.5 * tf.reduce_mean((self.reward - self.q) ** 2))
        self.ploss = policy_loss = -tf.reduce_mean(self.policy)
        self.absolute_errors = tf.abs(self.reward - self.q)

      self.policy_sub_opt = tf.train.AdamOptimizer(1e-3).minimize(policy_loss, var_list=get_vars('model/actor/sub'))
      self.policy_main_opt = tf.train.AdamOptimizer(1e-3).minimize(loss, var_list=get_vars('model/actor/main')+get_vars('model/actor/main2'))

      self.target_update = tf.group([tf.assign(v_targ, 0.001*v_targ + (1-0.001)*v_main)
                                for v_main, v_targ in zip(get_vars('model'), get_vars('target'))])

      target_init = tf.group([tf.assign(v_targ, v_main)
                              for v_main, v_targ in zip(get_vars('model'), get_vars('target'))])

      self.sess.run(tf.global_variables_initializer())
      # self.sess.run(target_init)
    def preproc(self):
        self.dat = df = pd.read_csv(self.path)
        s = np.asanyarray(ta.stoch(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1)) - np.asanyarray(ta.stoch_signal(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1))
        x = np.asanyarray(ta.daily_return(df["Close"])).reshape((-1,1))
        m = np.asanyarray(ta.macd_diff(df["Close"])).reshape((-1,1))
        cross1 = np.asanyarray(ta.ema(self.dat["Close"],12)).reshape((-1, 1)) - np.asanyarray(ta.ema(self.dat["Close"],5)).reshape((-1, 1))
        trend = np.asanyarray(df[["Close"]]) - np.asanyarray(ta.ema(self.dat["Close"],50)).reshape((-1, 1))
        # x = s
        x = np.concatenate([s,x,cross1,trend,m], 1)
        y = np.asanyarray(self.dat[["Open"]])

        gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x, y, self.window_size)
        self.x = []
        self.y = []
        for i in gen:
            self.x.extend(i[0].tolist())
            self.y.extend(i[1].tolist())
        self.x = np.asanyarray(self.x)[100:] #.reshape((-1, self.window_size, x.shape[-1]))
        self.y = np.asanyarray(self.y)[100:]

        self.df = self.x[-self.STEP_SIZE:]
        self.trend = self.y[-self.STEP_SIZE:]

    def _construct_memories_and_train(self,i, replay=None):
      try:
          tree_idx, replay = self.memory.sample(128)
      except:
          self.memory = Memory(5000)

      states = np.array([a[0][0] for a in replay])
      new_states = np.array([a[0][3] for a in replay])
      actions = np.array([a[0][1] for a in replay]).reshape((-1, 2))
      rewards = np.array([a[0][2] for a in replay]).reshape((-1, 1))
      done = np.array([a[0][-1] for a in replay]).reshape((-1, 1))

      # target_q = self.sess.run(self.target_q, feed_dict={self.new_state:new_states}).reshape((-1,1))
      q = self.sess.run(self.q,feed_dict={self.state:states,self.action:actions})
      new_q = self.sess.run(self.q_pi,feed_dict={self.state:new_states})
      new_target_q = self.sess.run(self.target_q_pi, feed_dict={self.new_state:new_states})

      for I in range(128):
        q[I,np.argmax(actions[I])] = rewards[I] + done[I] * 0.99 * new_target_q[I,np.argmax(new_q[I])]
      # print(q)

      step_ops = [self.absolute_errors,self.policy_main_opt]
      for _ in range(1):
        absolute_errors,_ = self.sess.run(step_ops, feed_dict={self.state: states, self.new_state: new_states, self.reward: q,self.action:actions})
      if i > 10:
        if (i+1) % 2 == 0:
              loss,_ = self.sess.run([self.loss,self.policy_sub_opt], feed_dict={self.state: states,self.reward: q,self.action:actions})
              # print(loss)
          # print([rewards,loss])
      # self.sess.run(self.target_update)
      ae = []
      for i in absolute_errors:
        ae.append(np.mean(i))
      ae = np.array(ae)
      self.memory.batch_update(tree_idx, ae)

    def _select_action(self, state, i):
        # self.policy_out
        prediction,q = self.sess.run([self.out,self.q_pi], feed_dict={self.state: [state]})
        prediction,q = prediction[0],q[0]

        if self.noise > self.noise_min:
          self.noise *= 0.9999
        noise = 0. if (i + 1) % 5 == 0 else self.noise
        # print(prediction)
        prediction += (noise * np.random.randn(2))
        prediction = np.clip(prediction, -1, 1)
        q *= prediction
        # prediction = q
        action = np.argmax(q)
        self.pred = prediction

        return action

    def prob(self,history):
        prob = np.asanyarray(history)
        a = np.mean(prob == 0)
        b = np.mean(prob == 1)
        c = 1 - (a + b)
        prob = [a,b,c]
        return prob

    def discount_rewards(self, r, running_add):
        running_add = running_add * 0.99 + r
        return running_add
        
    def nstep(self,r):
        running_add = 0.0
        for t in range(len(r)):
            running_add += 0.99 * r[t]

        return running_add

    def run(self, spread=10, pip_cost=1000, los_cut=600, day_pip=20, n=10,step=100):
        spread = spread / pip_cost
        self.rand = np.random.RandomState()
        lc = los_cut / pip_cost
        for i in range(100000):
            # if (i - 1) % step == 0:
            # h = self.rand.randint(self.x.shape[0]-(self.STEP_SIZE+1))
            # self.df = self.x[h:h+self.STEP_SIZE]
            # self.trend = self.y[h:h+self.STEP_SIZE]
            done = 1.0
            position = 3
            pip = []
            provisional_pip = []
            running_add = 0.0
            total_pip = 0.0
            old_reword = 0.0
            states = []
            h_a = []
            h_r = []
            h_p = []
            old = np.asanyarray(0)
            self.history = []
            for t in  range(0, len(self.trend)-1):
                action = self._select_action(self.df[t],i)
                h_a.append(self.pred)
                self.history.append(action)
                
                states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,
                                    action,position,states,pip_cost,spread,total_pip,lc=min([lc/2,100/pip_cost]))
                h_p.append(position)
                reward =  (total_pip - old_reword) * 100
                old_reword = total_pip
                h_r.append(reward)

                # running_add = self.discount_rewards(reward,running_add)
                # r_d = int(running_add * (pip_cost / 100)) * 100
                # # r_d = int(np.mean(np.array(provisional_pip) > 0)*100) if len(provisional_pip) != 0 else 0.
                # if t == len(self.trend)-5:
                #     done = 0.
                # exp = self.df[t], h_a[t], r_d, self.df[t+1], done
                # self.memory.store(exp)
            for t in range(0, len(self.trend)-1):
                tau = t - n + 1
                if tau >= 0:
                    rewards = self.nstep(h_r[tau+1:tau+n])
                    exp = self.df[t], h_a[t], int(rewards), self.df[t+1], done
                    self.memory.store(exp)

            try:
              self._construct_memories_and_train(i)
            except:
              traceback.print_exc()

            if i % 1000 == 0:
              clear_output()

            if (i + 1) % 5 == 0:
                # loss = np.mean(ae)
                self.pip = np.asanyarray(provisional_pip) * pip_cost
                self.pip = [p if p >= -los_cut else -los_cut for p in self.pip]
                self.total_pip = np.sum(self.pip)
                mean_pip = self.total_pip / (t + 1)
                trade_accuracy = np.mean(np.asanyarray(self.pip) > 0)
                self.trade = trade_accuracy
                mean_pip *= day_pip
                prob = self.prob(self.history)
                position_prob = self.prob(h_p)

                # print("loss =", loss)
                # print("")
                print('action probability = ', prob)
                print("buy = ", position_prob[1], " sell = ", position_prob[-1])
                print('trade accuracy = ', trade_accuracy)
                print('epoch: %d, total rewards: %f, mean rewards: %f' % (i + 1, float(self.total_pip), float(mean_pip)))
                print("")

In [0]:
window_size = 30
path = "audpred1440.csv"
agent = Agent(path,window_size)

agent.run()

action probability =  [0.4789915966386555, 0.5210084033613446, 0.0]
buy =  0.4789915966386555  sell =  0.5210084033613445
trade accuracy =  0.45871559633027525
epoch: 5, total rewards: 16535.000000, mean rewards: 2778.991597

action probability =  [0.5798319327731093, 0.42016806722689076, 0.0]
buy =  0.5798319327731093  sell =  0.4201680672268907
trade accuracy =  0.4017857142857143
epoch: 10, total rewards: 12397.000000, mean rewards: 2083.529412

action probability =  [0.6134453781512605, 0.3865546218487395, 0.0]
buy =  0.6134453781512605  sell =  0.38655462184873945
trade accuracy =  0.45535714285714285
epoch: 15, total rewards: 8091.000000, mean rewards: 1359.831933

action probability =  [0.7563025210084033, 0.24369747899159663, 0.0]
buy =  0.7563025210084033  sell =  0.24369747899159666
trade accuracy =  0.38392857142857145
epoch: 20, total rewards: -7006.000000, mean rewards: -1177.478992

action probability =  [0.6638655462184874, 0.33613445378151263, 0.0]
buy =  0.663865546218

In [0]:
try:
    tree_idx, replay = agent.memory.sample(128)
except:
    agent.memory = Memory(5000)

states = np.array([a[0][0] for a in replay])
new_states = np.array([a[0][3] for a in replay])
actions = np.array([a[0][1] for a in replay]).reshape((-1, 2))
rewards = np.array([a[0][2] for a in replay]).reshape((-1, 1))
done = np.array([a[0][-1] for a in replay]).reshape((-1, 1))

# target_q = self.sess.run(self.target_q, feed_dict={self.new_state:new_states}).reshape((-1,1))
q = agent.sess.run(agent.q,feed_dict={agent.state:states,agent.action:actions})
new_q = agent.sess.run(agent.q_pi,feed_dict={agent.state:new_states})
new_target_q = agent.sess.run(agent.target_q_pi, feed_dict={agent.new_state:new_states})

for I in range(128):
  q[I,np.argmax(actions[I])] = rewards[I] + done[I] * 0.99 * new_target_q[I,np.argmax(new_q[I])]
# print(q)

In [0]:
q[0:10]

In [0]:
new_q[0:10]